## Uwaga

1. W oryginalnych pracach dotyczących niektórych z poniższych metod pojawiają się matematyczne "akrobacje", które polegają na przykład na definiowaniu "pierwiastka ze zdiagonalizowanej macierzy produktu zewnętrznego historii gradientów". Proszę absolutnie nie myśleć w ten sposób o tych optimizerach!


2. W tym notebooku wszystkie operacje na wektorach są zdefiniowane _element-wise_, zgodnie z regułami pakietu numpy. Na przykład:
    * kwadrat wektora to wektor kwadratów elementów (numpy.square)
    * pierwiastek z wektora to wektor pierwiastków z elementów (numpy.sqrt)
    * suma, różnica, iloraz, iloczyn - analogicznie
    * suma wektora i liczby oznacza dodanie tej liczby do każdej współrzędnej

## Adagrad

#### Parametry
* $\eta$ - learning rate (typowe wartości: od 0.001 do 0.01)
* $\epsilon$ - zapobiega dzieleniu przez zero (zazwyczaj: $10^{-8})$

#### Parametry wewnętrzne
* $h^{(t)}$ - wektor sumujący kwadraty gradientów do czasu $t$, wymiar taki sam jak $\theta$

#### Inicjalizacja

* $h^{(0)} = \mathbf{0}$

#### Update

1. $h^{(t+1)} = h^{(t)} + (\nabla L(\theta^{(t)}))^2$
2. $\theta^{(t+1)}=\theta^{(t)} - \eta \dfrac{\nabla L(\theta^{(t)})}{\sqrt{h^{(t+1)} + \epsilon}}$

#### Dyskusja

1. dzielenie ma __znormalizować__ gradient
2. normalizacja oddzielnie dla:
    * każdej współrzędnej gradientu
    * czyli dla każdej współrzędnej $\theta$
    * czyli dla każdego parametru modelu
3. $h$ jest sumą kwadratów, więc trzeba w mianowniku wziąć pierwiastek - bez pierwiastka działa słabo
4. __akumulacja__ kwadratów gradientów
    * $h$ to suma, a nie średnia
    * gdyby gradienty były stałe, mianownik rósłby proporcjonalnie do $\sqrt{t}$
    * w praktyce bardzo maleje $\Delta\theta$, model może __przestać się uczyć__

## RMSProp

#### Parametry
* $\eta$ - learning rate (typowe wartości: od 0.001 do 0.01)
* $\gamma$ - współczynnik średniej kroczącej (zazwyczaj: 0.9)
* $\epsilon$ - zapobiega dzieleniu przez zero (zazwyczaj: $10^{-8})$

#### Parametry wewnętrzne
* $h^{(t)}$ - wektor średniej kroczącej kwadratów gradientów do czasu $t$, wymiar taki sam jak $\theta$

#### Inicjalizacja

* $h^{(0)} = \mathbf{0}$

#### Update

1. $h^{(t+1)} = \gamma h^{(t)} + (1-\gamma)(\nabla L(\theta^{(t)}))^2$
2. $\theta^{(t+1)}=\theta^{(t)} - \eta \dfrac{\nabla L(\theta^{(t)})}{\sqrt{h^{(t+1)} + \epsilon}}$

#### Dyskusja

1. __cel__: usunąć problem Adagrad z szybko malejącym $\Delta\theta$
2. średnia krocząca (__moving average__)
    * podobna do zwykłej średniej
    * zapomina daleką przeszłość
    * nie wymaga pamiętania pełnej historii gradientów
    * __ograniczenie akumulowania__ gradientów do okienka, $\frac{1}{1-\gamma}$ (stałe) zamiast $\sqrt{t}$ (rosnące)
3. zaproponowane przez Hintona


## Adadelta

#### Parametry
* $\gamma$ - współczynnik średniej kroczącej (zazwyczaj: 0.95)
* $\epsilon$ - zapobiega dzieleniu przez zero, umożliwia rozpoczęcie uczenia (zazwyczaj: od $10^{-6}$ do $10^{-2}$)

#### Parametry wewnętrzne
* $h^{(t)}$ - wektor średniej kroczącej kwadratów gradientów do czasu $t$, wymiar taki sam jak $\theta$
* $d^{(t)}$ - wektor średniej kroczącej $\Delta\theta$ do czasu $t$, wymiar taki sam jak $\theta$
* trzeba też pamiętać $\theta^{(t-1)}$

#### Inicjalizacja

* $h^{(0)} = \mathbf{0}$
* $d^{(0)} = \mathbf{0}$

#### Update

1. $h^{(t+1)} = \gamma h^{(t)} + (1-\gamma)(\nabla L(\theta^{(t)}))^2$
2. $\theta^{(t+1)}=\theta^{(t)} - \sqrt{d^{(t)} + \epsilon} \dfrac{\nabla L(\theta^{(t)})}{\sqrt{h^{(t+1)} + \epsilon}}$
3. $d^{(t+1)} = \gamma d^{(t)} + (1-\gamma)(\theta^{(t+1)} - \theta^{(t)})^2$

#### Dyskusja

1. rozszerzenie RMSProp (ale wymyślone niezależnie jako poprawka Adagrad)
2. zastąpienie learning rate przez __wektor__
    * learning rate __proporcjonalny do__ średniego $\Delta\theta$
    * upodobnienie szybkości poprawek do poprawek
    * __eliminacja__ stałej uczenia z parametrów
3. ważna rola parametru $\epsilon$
    * nie tylko zapobiega dzieleniu przez zero
    * umożliwia rozpoczęcie uczenia - $d^{(1)}$ większe od zera
    * $\sqrt{\epsilon}$ wyznacza __dolne ograniczenie__ $\sqrt{d^{(t)} +\epsilon}$ - uczenie nie zatrzymuje się

## Adam Adaptive Moment Estimation

#### Parametry
* $\eta$ - learning rate (zazwyczaj: 0.001)
* $\beta_1$ - współczynnik średniej kroczącej pierwszego momentu (zazwyczaj: 0.9)
* $\beta_2$ - współczynnik średniej kroczącej pierwszego momentu (zazwyczaj: 0.999)
* $\epsilon$ - zapobiega dzieleniu przez zero (zazwyczaj: $10^{-8}$)

#### Parametry wewnętrzne
* $m^{(t)}$ - wektor średniej kroczącej pierwszego momentu gradientu do czasu $t$, wymiar taki sam jak $\theta$
* $v^{(t)}$ - wektor średniej kroczącej drugiego momentu gradientu do czasu $t$, wymiar taki sam jak $\theta$

#### Inicjalizacja

* $m^{(0)} = \mathbf{0}$
* $v^{(0)} = \mathbf{0}$

#### Update

1. $m^{(t+1)} = \beta_1 m^{(t)} + (1-\beta_1)(\nabla L(\theta^{(t)}))$
2. $v^{(t+1)} = \beta_2 v^{(t)} + (1-\beta_2)(\nabla L(\theta^{(t)}))^2$
3. $\widehat m = \dfrac{m^{(t+1)}}{1-\beta_1^{t+1}}$
4. $\widehat v = \dfrac{v^{(t+1)}}{1-\beta_2^{t+1}}$
5. $\theta^{(t+1)} = \theta^{(t)} - \eta\dfrac{\widehat m}{\sqrt{\widehat v} + \epsilon}$

Uwaga: $\beta^{t+1}$ to "$\beta$ do potęgi $t+1$", a nie $\beta$ w czasie $t+1$.

#### Dyskusja

* __gradient__ adaptowany
  * krocząca średnia gradientów (pierwszy moment) - tłumione oscylacje
  * krocząca średnia kwadratów gradientów (drugi moment) - normalizacja gradientów
* __inicjalizacja__ $v_0$ i $m_0$ na $0$
    * średnia krocząca __zbiasowana__ w kierunku zera
    * __przeciwdziałanie__
        * symbole z "daszkiem" wprowadzają poprawkę
        * im później (duże $t$), tym mniejsza poprawka
* dobrze dobrane parametry domyślne - zazwyczaj nie ma potrzeby ich modyfikacji
* jeden z __najlepszych__ optimizerów

## I wiele wiele innych

AdaMax, Nadam, AMSGrad, ...